In [1]:
import sys
import os
current = os.path.dirname(os.path.realpath("Single-House-Optimization.py"))
parent = os.path.dirname(current)
sys.path.append(parent+"\Functions")
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)

from itertools import permutations
from IPython.display import display, HTML

import numpy as np
import pandas as pd
from copy import deepcopy
from Merge import merge
from copy import deepcopy
from Logic import logic_rollout, action_rollout, pred_logic_rollout, print_price_summary, logic_series_print
from Battery import Battery
from DPModel import DPModel, DP, DP_stochastic

from P2P_Dynamics import EnergyMarket

In [2]:
Start = '2022-06-19 00:00:00'
End = '2022-06-19 23:00:00'
N = len(pd.date_range(Start, End,freq='H'))

dfA = merge('h16')
dfB = merge('h28')
dfC = merge('k28')

In [3]:
battery = Battery(max_capacity=13, max_charge=7)
series_DP_A = DP(Start, End, dfA, battery, byday=True, ints=True, degrade=False, verbose=True)
series_DP_B = DP(Start, End, dfB, Battery(max_capacity=13, max_charge=7,current_capacity=10), byday=True, ints=True, degrade=False, verbose=True)
series_DP_C = DP(Start, End, dfC, battery, byday=True, ints=True, degrade=False, verbose=True)

Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00


In [4]:
print_price_summary(series_DP_A, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  150.0  DKK
Total emissions for period:  13.0  kg

Number of kwh purchased in the period: 72.3
Number of kwh sold in the period: 6.5


In [5]:
print_price_summary(series_DP_B, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  70.0  DKK
Total emissions for period:  8.0  kg

Number of kwh purchased in the period: 38.7
Number of kwh sold in the period: 5.999999999999999


In [6]:
print_price_summary(series_DP_C, False)

The period is from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Cost for period:  173.0  DKK
Total emissions for period:  14.0  kg

Number of kwh purchased in the period: 80.10000000000001
Number of kwh sold in the period: 0.0


In [7]:
class DP_central(DPModel):
    def __init__(self, Start, End, houses, battery,degrade=False,ints=False,acts=None,acts_range=None,
                 furthers=None,traj=None, traj_range=None,max_number_states=200): 
        super().__init__(Start, End, merge(houses[0]), battery,degrade,ints,acts,acts_range)
        self.houses = houses
        
        self.furthers = furthers
        
        self.traj = traj
        self.traj_range = traj_range
        
        self.max_number_states = max_number_states
        
        temp = pd.DataFrame(columns=houses)
        for i in range(len(houses)):
            temp[houses[i]] = merge(houses[i]).loc[Start:End]["yield"]

        self.yields = temp
    
        #Compute state space once
        if ints:
            states = np.arange(0, battery.max_capacity+1, 1)
        else:
            states = np.round(np.arange(0.0, battery.max_capacity+0.01, 0.1),2)
        
        self.states = [states for _ in range(len(houses))]
        
        if self.ints and self.furthers is not None:
            for i in range(len(houses)):
                temp = self.states[i]
                self.states[i] = temp[temp%self.furthers[i][0]==self.furthers[i][1]]

        self.s = np.array(np.meshgrid(*self.states)).T.reshape(-1,3)
        self.s = self.s.tolist()
        self.s = [tuple(x) for x in self.s]
    
    def f(self, x, u, w, k):
        ogbat = self.battery
        self.battery = deepcopy(ogbat)
        
        res = []
        for i in range(len(self.houses)):
            self.battery.current_capacity = x[i]
            self.battery.charge(u[i], degrade=self.degrade)

            if self.ints:
                self.battery.current_capacity = int(self.battery.current_capacity)
            
            res.append(self.battery.get_current_capacity())
            
        self.battery = ogbat
        
        return tuple(res)
    
    def g(self, x, u, w, k):
        yields = self.get_yield(k)
        surpluses = [yields[i]-u[i] for i in range(len(u))]
        
        fee = 1 #transmission fee
        
        participants = {self.houses[i]: surpluses[i] for i in range(len(u))}
    
        em = EnergyMarket(participants, self.sp[k], self.sp[k]+fee)
        
        dic = em.get_total_costs()
        
        return sum([dic[house] for house in self.houses])
    
    def S(self, k):
        if (self.traj is not None) and (self.traj_range is not None):
            for i in range(len(self.houses)):
                if self.traj_range[i]==0.0:
                    self.states[i] = self.traj[k][i]
                
            temp = np.array(np.meshgrid(*self.states)).T.reshape(-1,3)
            temp = temp.tolist()
            temp = [tuple(x) for x in temp]
            
            return temp
        
        return self.s
    
    def A(self, x, k):
        states=[]
        acts = self.acts
        self.acts=None
        for i in range(len(x)):
            temp = super().A(x[i], k)
            states.append(temp)
            
        self.acts=acts
        
        if self.ints:
            states = [states[i][np.array(states[i],dtype=int)==states[i]] for i in range(len(x))]
            
        if self.acts is not None and self.acts_range is not None:
            for i in range(len(x)):
                if self.acts_range[i]!=0.0:
                    below =states[i][states[i]<=self.acts[k][i]][-int((self.acts_range[i]+0.1)*10):]
                    above =states[i][states[i]>self.acts[k][i]][:int(self.acts_range[i]*10)]

                    states[i] = np.append(below,above)
                else:
                    states[i] = self.acts[k][i]
        
        if self.ints and self.furthers is not None:
            states = [states[i][(states[i]+x[i])%self.furthers[i][0]==self.furthers[i][1]] for i in range(len(x))]
            
        actions = np.array(np.meshgrid(*states)).T.reshape(-1,3)
        
        idx = np.round(np.linspace(0, len(actions) - 1, self.max_number_states)).astype(int)

        return actions[idx] if len(actions)>self.max_number_states else actions
    
    def get_yield(self,k):
        return self.yields.iloc[k].to_numpy()

In [8]:
def tup_sub(tup1,tup2):
    res = [int((tup1[i]-tup2[i])*10)/10 for i in range(len(tup1))] 
    return tuple(res)

def tup_add(tup1,tup2):
    res = [int((tup1[i]+tup2[i])*10)/10 for i in range(len(tup1))] 
    return tuple(res)

def correct_traj_acts(x0,traj,acts):
    traj[0]=x0
    for i in range(len(acts)):
        traj[i+1]=tup_add(traj[i],acts[i])
    
    return traj

def policy_rollout(model, pi, x0):
    cost = 0
    surpluses = []
    J, x, trajectory, actions = 0, x0, [x0], []
    for k in range(model.N):
        u = pi(x, k)
        price = model.g(x, u , True, k)
        surpluses.append([model.get_yield(k)[i]-u[i] for i in range(len(u))])
        J+=price

        x = model.f(x, u, True, k)
        trajectory.append(x) # update the list of the trajectory
        actions.append(u) # update the list of the actions
        
    J += model.gN(x)
    return J, trajectory, actions, np.array(surpluses)

In [9]:
class DP_P2P:
    def __init__(self, start_time, end_time, houses, battery):
        if len(houses)<=1:
            raise Exception("P2P requires more than one house!")
        if False in [house in ["k28", "h16", "h22", "h28", "h32"] for house in houses]:
            raise Exception('All houses should be either "k28", "h16", "h22", "h28", or "h32"')
        if type(battery) is not Battery:
            raise Exception("battery must be a Battery class instance!")
            
        self.start_time = start_time
        self.end_time = end_time
        self.houses = houses
        
        self.battery = battery
        self.N = len(pd.date_range(start=start_time,end=end_time,freq="h"))
        self.results = None
        self.results_ord = None
        self.all_results = None
        self.all_results_ord = None
        
        self.nf = None
        self.all_nf = None
        
        self.all_costs = None
        
        self.perms_names = None
        self.this_perm = None
        self.this_perm_name = None
        
        all_houses = ["h16", "h22", "h28", "h32", "k28"] 
        self.all_houses = all_houses
        
        houses_ord = []
        for house in self.all_houses:
            if house in self.houses:
                houses_ord.append(house)
                
        self.houses_ord = houses_ord
        
        merged = merge(houses[0])
        self.sp = merged.loc[Start:End]["SpotPriceDKK"]/1000
        
        idx = []
        for i in range(len(all_houses)):
            for j in range(len(self.houses)+1):
                if j>=len(self.houses):
                    idx.append(-1)
                    break
                
                if all_houses[i]==self.houses[j]:
                    idx.append(j)
                    break
        self.idx = idx
    
    def run(self, j, Start, End, x0, x0_int, max_number_states, trajectory=None, actions=None, ints=False):
        if ints:    
            furthers=[[1,0]]
            for t in range(len(self.houses)-1):
                furthers.append([self.battery.max_capacity+1,x0_int[t+1]])

            DPP2P= DP_central(Start, End, self.houses, deepcopy(self.battery),
                              degrade=False,ints=True,acts=None,acts_range=None,
                              furthers=furthers,traj=None,traj_range=None, max_number_states=max_number_states)
            _, pi = DP_stochastic(DPP2P)

            J, trajectory, actions, surpluses = policy_rollout(DPP2P, pi=lambda x, k: pi[k][x], x0=x0_int)

            if x0 != x0_int:
                trajectory = correct_traj_acts(x0,trajectory,actions)

        elif (trajectory is not None) and (actions is not None):
            traj_range = [0 if t!=j+1 else -1 for t in range(len(self.houses))]  
            acts_range = [0 if t!=j+1 else self.battery.max_charge for t in range(len(self.houses))]

            DPP2P= DP_central(Start, End, self.houses, deepcopy(self.battery),
                              degrade=False,ints=False,acts=actions,acts_range=acts_range,
                              furthers=None,traj=trajectory,traj_range=traj_range, max_number_states=max_number_states)
            _, pi = DP_stochastic(DPP2P)

            J, trajectory, actions, surpluses = policy_rollout(DPP2P, pi=lambda x, k: pi[k][x], x0=x0)

        else:
            raise Exception("Must have either furthers or trajectory and actions as input!")

        return J, trajectory, actions, surpluses
        
    def P2P_sol(self, x0, max_number_states=20, brute_push = False, byday=True, verbose=True):
        N=self.N

        all_actions = []
        all_surpluses = np.ones((0,len(self.houses)))

        Start_i = self.start_time
        x0_i = x0
        x0_int_i = tuple([int(x0[i]) for i in range(len(x0))])

        num_loops = int(np.ceil(N/24)) if byday else 1
        remainder = N%24
        length = 24 if byday else N
        for i in range(num_loops):
            if byday and i == num_loops-1:
                length = length if remainder == 0 else remainder

            End_i = pd.date_range(start=Start_i,periods=length,freq="h")[-1]

            if verbose:
                print(f"Period from {Start_i} to {End_i}")
            
            furthers=[[1,0]]
            for t in range(len(self.houses)-1):
                furthers.append([self.battery.max_capacity+1,x0_int_i[t+1]])
            
            for j in range(len(self.houses)):
                if j==0:
                    J, trajectory, actions, surpluses = self.run(j, Start_i, End_i, x0_i, x0_int_i, max_number_states, 
                                                                 trajectory=None, actions=None, ints=True)
                        
                J, trajectory, actions, surpluses = self.run(j, Start_i, End_i, x0_i, x0_int_i, max_number_states, 
                                                             trajectory=trajectory, actions=actions, ints=False)
       
            trajectory_og, actions_og, surpluses_og = deepcopy(trajectory), deepcopy(actions), np.copy(surpluses)
           
            if brute_push:
                while True:
                    if verbose:
                        print("Brute forcing another round")
                    for j in range(len(self.houses)):       
                        temp, trajectory, actions, surpluses = self.run(j, Start_i, End_i, x0_i, x0_int_i, max_number_states, 
                                                                        trajectory=trajectory, actions=actions, ints=False)

                    J_prev=J
                    J = temp
                    if verbose:
                        print(f"J_prev = {J_prev}, J = {J}")
                        print()
                    if J>=J_prev:
                        trajectory, actions, surpluses = trajectory_og, actions_og, surpluses_og
                        break

                    trajectory_og, actions_og, surpluses_og = deepcopy(trajectory), deepcopy(actions), np.copy(surpluses)
                
            
            all_actions = all_actions + actions
            all_surpluses = np.append(all_surpluses,surpluses,axis=0)

            Start_i= pd.date_range(start=End_i,periods=2,freq="h")[-1]
            
            x0_i = trajectory[-1]
            x0_int_i = tuple([int(x0_i[t]) for t in range(len(x0_i))])

        self.results = (all_actions,all_surpluses)
        
        
        all_surpluses_ord = np.copy(all_surpluses)
        all_actions_ord = np.array(all_actions)
        
        i=0
        for j in range(len(self.idx)):
            if self.idx[j]!=-1:
                all_surpluses_ord[:,i] = all_surpluses[:,self.idx[j]]
                all_actions_ord[:,i] = np.array(all_actions)[:,self.idx[j]]
                i+=1
        
        f = lambda l: tuple(l)
        all_actions_ord = list(map(f,all_actions_ord))
        
        self.results_ord =(all_actions_ord,all_surpluses_ord)
        
        return self.results_ord
    
    def cost_matrix(self):
        if self.results is None:
            return None
        
        _,surpluses_ord = self.results_ord
        
        nf = pd.DataFrame()
        for i,house in enumerate(self.houses_ord):
            nf[house] = surpluses_ord[:,i]
                
        (all_actions_ord,all_surpluses_ord) = self.results_ord
            
        new = pd.DataFrame()
        for i in range(len(nf)):
            thing = pd.DataFrame(EnergyMarket(nf.iloc[i].to_dict(),self.sp[i],self.sp[i]+1).get_total_costs(), index=[0])
            new = pd.concat([new,thing],ignore_index=True)
            
        for house in self.houses_ord:
            nf['O'+house] = new[house].to_list()
        
        for house in self.houses_ord:
                nf['cumm_O'+house] = nf['O'+house].cumsum()
         
        self.nf = nf
        
        return nf
    
    def total_cost(self):
        if self.nf is None:
            return None
        
        return sum([self.nf['cumm_O'+house][len(self.nf)-1] for house in self.houses])
    
    def all_sol(self, x0, max_number_states=20, brute_push = False, byday=True, verbose=True):
        perms = list(permutations([i+1 for i in range(len(self.houses))]))
        
        house_rewrites = [[self.houses_ord[i-1] for i in perms[j]] for j in range(len(perms))]
        
        
        perms_names = [''.join(str(i) for i in perms[j]) for j in range(len(perms))]
        
        self.perms_names = perms_names
        
        for i, houses in enumerate(house_rewrites):
            if tuple(houses)==tuple(self.houses):
                this_idx = i
                this_perm = perms[this_idx]
                this_perm_name = perms_names[this_idx]
                
                break
                
        self.this_perm = this_perm
        self.this_perm_name = this_perm_name
               
        x0_ord = tuple([x0[(len(houses)-i)] for i in this_perm])
        
        all_results = []
        all_results_ord = []
        all_nf = []
        all_costs = []
            
        if verbose:
            print(f"This permutation is called '{this_perm_name}'. Starting with permutation '{perms_names[0]}'")
            print()
            
        for i in range(len(house_rewrites)):
            if verbose:
                print(f"Permutation {perms_names[i]}, ({i+1}/{len(perms_names)})")
            
            x0 = tuple([x0_ord[j-1] for j in perms[i]])
            self.houses = house_rewrites[i]
            idx = []
            for i in range(len(self.all_houses)):
                for j in range(len(self.houses)+1):
                    if j>=len(self.houses):
                        idx.append(-1)
                        break

                    if self.all_houses[i]==self.houses[j]:
                        idx.append(j)
                        break
            self.idx = idx
            
            self.P2P_sol(x0, max_number_states=max_number_states, brute_push = brute_push, byday=byday, verbose=verbose)
            self.cost_matrix()
            
            all_results.append(self.results)
            all_results_ord.append(self.results_ord)
            all_nf.append(self.nf)
            all_costs.append(self.total_cost())

            if verbose:
                print()
        
        self.houses = house_rewrites[this_idx]
        self.all_results = all_results
        self.all_results_ord = all_results_ord
        self.all_nf = all_nf
        self.all_costs = all_costs
        
        idx = []
        for i in range(len(self.all_houses)):
            for j in range(len(self.houses)+1):
                if j>=len(self.houses):
                    idx.append(-1)
                    break

                if self.all_houses[i]==self.houses[j]:
                    idx.append(j)
                    break
        self.idx = idx
        
        if verbose:
            best_idx = np.argmin(all_costs)
            print("Done!")
            print()
            print(f"Best found permutation was {perms_names[best_idx]} with cost {all_costs[best_idx]}")

In [10]:
bat = Battery(max_capacity=13,max_charge=7)
max_number_states=20
x0 = (0,10,0)

In [11]:
stuff = DP_P2P(Start,End,["h16","h28","k28"],bat)
stuff.all_sol(x0, max_number_states=20, brute_push = True, byday=True, verbose=True)

This permutation is called '123'. Starting with permutation '123'

Permutation 123, (1/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Brute forcing another round
J_prev = 350.25548223316997, J = 350.03875662196003

Brute forcing another round
J_prev = 350.03875662196003, J = 350.11097787505986


Permutation 132, (2/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Brute forcing another round
J_prev = 349.81309859976, J = 347.11487387394

Brute forcing another round
J_prev = 347.11487387394, J = 346.6123399892

Brute forcing another round
J_prev = 346.6123399892, J = 346.71954607550003


Permutation 213, (3/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00
Brute forcing another round
J_prev = 344.63385700607, J = 335.33550202874994

Brute forcing another round
J_prev = 335.33550202874994, J = 332.26266570459

Brute forcing another round
J_prev = 332.26266570459, J = 333.07762119245


Permutation 231, (4/6)
Period from 2022-06-19 00:00:00 to 2022-06-19 23:00:00

In [12]:
for i in range(len(stuff.all_nf)):
    print(f"This is the cost matrix for sol with permutation {stuff.perms_names[i]}")
    display(HTML(stuff.all_nf[i]._repr_html_()))
    print(f"Total cost this permutation = {stuff.all_costs[i]}")
    print()
    print()
    print()

This is the cost matrix for sol with permutation 123


,h16,h28,k28,Oh16,Oh28,Ok28,cumm_Oh16,cumm_Oh28,cumm_Ok28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330
2,-4.2,-10.3,-5.6,9.371460,22.982391,12.495280,36.134678,20.317555,44.044611
3,-3.8,4.2,-4.1,3.951886,-7.830974,4.263877,40.086564,12.486582,48.308487
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,47.295696,18.178002,56.086762
5,-10.4,-3.1,-4.6,18.984472,5.658833,8.396978,66.280168,23.836835,64.483740
6,3.5,-3.4,-3.4,-5.417072,2.846684,2.846684,60.863096,26.683519,67.330424
7,-10.1,-2.9,-4.3,17.505926,5.026454,7.453018,78.369022,31.709973,74.783442
8,3.7,-3.6,-3.7,-4.673743,2.424420,2.491765,73.695279,34.134393,77.275207
9,-10.2,-5.6,-6.0,17.353770,9.527560,10.208100,91.049049,43.661953,87.483306


Total cost this permutation = 350.03875662196003



This is the cost matrix for sol with permutation 132


,h16,h28,k28,Oh16,Oh28,Ok28,cumm_Oh16,cumm_Oh28,cumm_Ok28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330
2,-4.2,-10.3,-5.6,9.371460,22.982391,12.495280,36.134678,20.317555,44.044611
3,-3.8,4.2,-4.1,3.951886,-7.830974,4.263877,40.086564,12.486582,48.308487
4,-3.8,-3.0,-4.1,7.209132,5.691420,7.778274,47.295696,18.178002,56.086762
5,-10.4,-3.1,-4.6,18.984472,5.658833,8.396978,66.280168,23.836835,64.483740
6,3.5,-3.4,-3.4,-5.417072,2.846684,2.846684,60.863096,26.683519,67.330424
7,-10.1,-2.9,-6.5,17.505926,5.026454,11.266190,78.369022,31.709973,78.596614
8,3.7,-3.6,-3.7,-4.673743,2.424420,2.491765,73.695279,34.134393,81.088379
9,-10.2,0.0,-4.9,17.353770,-0.000000,8.336615,91.049049,34.134393,89.424994


Total cost this permutation = 346.61233998920005



This is the cost matrix for sol with permutation 213


...

Total cost this permutation = 339.48813418455995



This is the cost matrix for sol with permutation 312


,h16,h28,k28,Oh16,Oh28,Ok28,cumm_Oh16,cumm_Oh28,cumm_Ok28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847
1,-7.1,-0.3,-6.3,17.135211,0.724023,15.204483,31.107356,-2.664835,31.549330
2,-2.9,-10.3,-5.6,6.470770,22.982391,12.495280,37.578126,20.317555,44.044611
3,-4.1,4.2,-4.1,4.263877,-8.111766,4.263877,41.842003,12.205790,48.308487
4,-3.0,-3.0,-4.1,5.691420,5.691420,7.778274,47.533423,17.897210,56.086762
5,-4.5,-2.7,-10.5,8.214435,4.928661,19.167015,55.747858,22.825871,75.253776
6,-3.7,-3.7,3.8,3.097862,3.097862,-5.894310,58.845720,25.923733,69.359466
7,-2.6,-1.3,-11.3,4.506476,2.253238,19.585838,63.352196,28.176971,88.945304
8,-4.1,-4.1,4.2,2.761145,2.761145,-5.252910,66.113341,30.938116,83.692394
9,-8.4,-0.7,-8.4,14.291340,1.190945,14.291340,80.404681,32.129061,97.983734


Total cost this permutation = 338.10693587668004



This is the cost matrix for sol with permutation 321


,h16,h28,k28,Oh16,Oh28,Ok28,cumm_Oh16,cumm_Oh28,cumm_Ok28
0,-5.5,2.2,-7.3,13.972145,-3.388858,16.344847,13.972145,-3.388858,16.344847
1,-5.3,-0.3,-6.3,12.791073,0.724023,15.204483,26.763218,-2.664835,31.549330
2,-5.7,-10.3,-5.6,12.718410,22.982391,12.495280,39.481629,20.317555,44.044611
3,-4.1,4.2,-4.1,4.263877,-8.111766,4.263877,43.745505,12.205790,48.308487
4,-2.0,-3.0,-4.1,3.794280,5.691420,7.778274,47.539785,17.897210,56.086762
5,-4.5,-2.7,-10.5,8.214435,4.928661,19.167015,55.754220,22.825871,75.253776
6,-3.7,-3.7,3.8,3.097862,3.097862,-5.894310,58.852082,25.923733,69.359466
7,-2.6,-1.3,-11.3,4.506476,2.253238,19.585838,63.358558,28.176971,88.945304
8,-4.1,-4.1,4.2,2.761145,2.761145,-5.252910,66.119704,30.938116,83.692394
9,-8.4,-0.7,-8.4,14.291340,1.190945,14.291340,80.411043,32.129061,97.983734


Total cost this permutation = 342.33367990617006





In [13]:
df = pd.DataFrame()
for i in range(len(stuff.all_results_ord)):
    df["actions"+stuff.perms_names[i]] = stuff.all_results_ord[i][0]
df

,actions123,actions132,actions213,actions231,actions312,actions321
0,"(0.0, -7.0, -0.0)","(0.0, -7.0, -0.0)","(-0.0, -7.0, -0.0)","(-0.0, -7.0, -0.0)","(-0.0, -7.0, 0.0)","(-0.0, -7.0, 0.0)"
1,"(0.0, -3.0, -0.0)","(0.0, -3.0, -0.0)","(-0.0, -3.0, -0.0)","(-0.0, -3.0, -0.0)","(1.8, -3.0, 0.0)","(-0.0, -3.0, 0.0)"
2,"(0.0, 7.0, -0.0)","(0.0, 7.0, -0.0)","(1.5, 7.0, -0.0)","(1.1, 7.0, -0.0)","(-1.3, 7.0, 0.0)","(1.5, 7.0, 0.0)"
3,"(0.0, -7.0, -0.0)","(0.0, -7.0, -0.0)","(0.3, -7.0, -0.0)","(0.2, -7.0, -0.0)","(0.3, -7.0, 0.0)","(0.3, -7.0, 0.0)"
4,"(0.0, -0.0, -0.0)","(0.0, -0.0, -0.0)","(-0.9, 7.0, 2.9)","(2.4, 7.0, -0.0)","(-0.8, -0.0, 0.0)","(-1.8, -0.0, 0.0)"
5,"(7.0, 0.4, 1.1)","(7.0, 0.4, 1.1)","(0.8, -7.0, 0.7)","(0.8, -7.0, 0.7)","(1.1, -0.0, 7.0)","(1.1, -0.0, 7.0)"
6,"(-7.0, 0.8, 0.2)","(-7.0, 0.8, 0.2)","(-0.3, 7.0, -3.0)","(-3.9, 7.0, 0.4)","(0.2, 1.1, -7.0)","(0.2, 1.1, -7.0)"
7,"(7.0, 0.5, 0.0)","(7.0, 0.5, 2.2)","(1.3, -7.0, 0.2)","(1.4, -7.0, 0.2)","(-0.5, -1.1, 7.0)","(-0.5, -1.1, 7.0)"
8,"(-7.0, 1.0, 0.9)","(-7.0, 1.0, 0.9)","(6.2, 7.0, 3.3)","(4.2, 7.0, 5.8)","(0.8, 1.5, -7.0)","(0.8, 1.5, -7.0)"
9,"(7.0, 4.9, 4.6)","(7.0, -0.7, 3.5)","(-0.0, -4.0, 1.7)","(-0.0, -4.0, 1.8)","(5.2, 0.0, 7.0)","(5.2, 0.0, 7.0)"
